In [51]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

In [67]:
base = 'https://www.life-global.org'
url = 'https://www.life-global.org/allcourses?page=1'

def getSoup(uri):
    driver_path = 'chromedriver.exe'
    driver = webdriver.Chrome()
    driver.get(uri)
    time.sleep(3)
    html = driver.page_source
    driver.quit()
    soup = bs(html, 'html.parser')
    return soup
"""
Esta función estaba pensada para recuperar el html relevante a las lecciones de los cursos, pero desafortunadamente esa parte de lá página es
dinámica y funciona de manera extraña dado que no existe ningún botón ni enlace que podamos seguir. Posiblemente sea un script pero no pudimos obtener acceso
a ellos. Ante esto, nos dimos cuenta que la estructura de las lecciones de los cursos es la misma siempre, así que nos tomamos la libertad de guardarlos de manera manual.
"""
def getSoupAndFlex(uri):
    print(uri)
    driver_path = 'chromedriver.exe'
    driver = webdriver.Chrome()
    driver.get(uri)
    time.sleep(3)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    wait = WebDriverWait(driver,10)
    wait.until(EC.visibility_of_element_located((By.XPATH,"//li[contains(@class, 'ui-core-emotion-cache-34o57r')]")))

    element = driver.find_element(By.XPATH,"//li[contains(@class, 'ui-core-emotion-cache-34o57r')]")

    actions = ActionChains(driver)
    actions.click(element).perform()
    html2 = driver.page_source
    html2 = html = driver.page_source
    soupLesson = bs(html2, 'html.parser')
    
    driver.quit()
    
    return soup, soupLesson


In [32]:
def getCatAndLinks(soup, base):
    certificat = soup.find_all('h2', class_="ui-core-emotion-cache-1htcy9t")
    certifications = []
    for cer in certificat:
        certifications.append(cer.text)
    links = []
    linki = soup.find_all('span', class_="ui-core-emotion-cache-4233sn") 
    linki = linki[2:]
    linki = linki[:5]
    for link in linki:
        links.append(link.get("href"))
    cerlinks = [base+cer for cer in links]
    print(certifications)
    print(cerlinks)
    return certifications, cerlinks


In [33]:
def getCatInfo(names, links, base):
    courses = {}
    for n in names:
        courses[n] = []
    couLinks = {}
    for n in names:
        couLinks[n] = []
    i = 0
    for link in links:

        name = names[i]
        soup = getSoup(link)

        coursi = soup.find_all('h1', class_="ui-core-emotion-cache-g77xg1")
        temp = []
        for co in coursi:
            temp.append(co.text)
        courses[name] = temp

        linki = soup.find_all("span", class_="ui-core-emotion-cache-4233sn")
        temp = []
        for li in linki:
            temp.append(li.get("href"))
        temp = [base+li for li in temp]
        temp = temp[2:]
        temp = temp[:-16]
        couLinks[name] = temp

        i += 1
    return courses, couLinks



In [64]:
def getCourseAndLessonsInfo(dicNombres, dicLinks):
    cat = dicNombres.keys()
    coursesInfo = []
    lessonsInfo = []
    for ca in cat:
        links = dicLinks[ca]
        i = 0
        for co in dicNombres[ca]:
            dicto = {
                "Name":co,
                "Duration":"",
                "Pace":"",
                "Language":"",
                "Author":"",
                "Code":"",
                "Description":"",
                "Category":"",
                "Link":links[i]
            }
            coursesInfo.append(dicto)
            i +=1

    for j in range(0,len(coursesInfo)):
        cu = coursesInfo[j]
        soup = getSoupAndFlex(cu["Link"])
        info = soup[0].find_all("div", class_ = "ui-core-emotion-cache-1ui1n6c")
        cu["Duration"] = info[0].text
        cu["Pace"] = info[1].text
        cu["Category"] = info[2].text
        cu["Language"] = info[3].text
        cu["Author"] = info[4].text
        cu["Code"] = info[5].text
        cu["Description"] = soup[0].find("div", class_ = "ui-core-emotion-cache-8gtjff").text
        
        #Recuperamos la información de las lecciones pero como conservan una estructura similar pordemos fijar la manera en que se guarda
        info2 = soup[1].find_all("div", class_= "ui-core-emotion-cache-1yn1t3u")
        info3 = soup[1].find_all("div", class_= "ui-core-emotion-cache-16o4g8j")

        try:
            lesson1 = {
                "Type": info2[0].text,
                "Name": info3[0].text,
                "Course": cu["Name"]
            }
            lessonsInfo.append(lesson1)

            lesson2 = {
                "Type": info2[1].text,
                "Name": info3[1].text,
                "Course": cu["Name"]
            }
            lessonsInfo.append(lesson2)

            lesson3 = {
                "Type": info2[1].text,
                "Name": info3[2].text,
                "Course": cu["Name"]
            }
            lessonsInfo.append(lesson3)

            lesson4 = {
                "Type": info2[1].text,
                "Name": info3[3].text,
                "Course": cu["Name"]
            }
            lessonsInfo.append(lesson4)

            lesson5 = {
                "Type": info2[1].text,
                "Name": info3[4].text,
                "Course": cu["Name"]
            }
            lessonsInfo.append(lesson5)
        except:
            pass


    return coursesInfo, lessonsInfo


In [35]:
languageAll = ["https://www.life-global.org/allcourses?page=1", "https://www.life-global.org/fr/allcourses?page=1", "https://www.life-global.org/ar/allcourses?page=1", "https://www.life-global.org/hi/allcourses?page=1", "https://www.life-global.org/pt/allcourses?page=1", "https://www.life-global.org/es/allcourses?page=1", "https://www.life-global.org/zh-Hans/allcourses?page=1", "https://www.life-global.org/id/allcourses?page=1"]
languageBase = ["https://www.life-global.org/", "https://www.life-global.org/fr", "https://www.life-global.org/ar", "https://www.life-global.org/hi", "https://www.life-global.org/pt", "https://www.life-global.org/es", "https://www.life-global.org/zh-Hans","https://www.life-global.org/id"]

def getAll(urlAll, urlBase):

    soupAll = getSoup(urlAll)

    cate = getCatAndLinks(soupAll, urlBase)
    catenames = cate[0]
    catelinks = cate[1]

    catCourses = getCatInfo(catenames, catelinks, urlBase)
    courNames = catCourses[0]
    courLinks = catCourses[1]

    coursesLessons = getCourseAndLessonsInfo(courNames, courLinks)
    coursesLa = coursesLessons[0]
    lessonsLa = coursesLessons[1]

    return catenames, coursesLa, lessonsLa


In [96]:
df_categories = None
df_courses = None
df_lessons = None


language =  getAll(languageAll[1], languageBase[1])

if df_categories == None:
    df_categories = pd.DataFrame(language[0], columns=['Name'])
else:
    new = pd.DataFrame(language[0], columns=['Name'])
    df_categories = df_categories.append(new, ignore_index = True)

if df_courses == None:
    df_courses = pd.DataFrame(language[1])
else:
    df_courses = df_courses.append(language[1], ignore_index = True)

if df_lessons == None:
    df_lessons = pd.DataFrame(language[2])
else:
    df_lessons = df_lessons.append(language[2], ignore_index = True)
    

['Communication', 'Créer / Innover', 'La Finance', 'Commercialisation', 'Opérations']
['https://www.life-global.org/fr/fr/categorylist/1-communication', 'https://www.life-global.org/fr/fr/categorylist/2-cr%C3%A9er---innover', 'https://www.life-global.org/fr/fr/categorylist/3-la-finance', 'https://www.life-global.org/fr/fr/categorylist/4-commercialisation', 'https://www.life-global.org/fr/fr/categorylist/5-op%C3%A9rations']
https://www.life-global.org/fr/fr/course/127-des-pr%C3%A9sentations-efficaces
https://www.life-global.org/fr/fr/course/125-pr%C3%A9senter-des-donn%C3%A9es
https://www.life-global.org/fr/fr/course/126-les-e-mails-professionnels
https://www.life-global.org/fr/fr/course/123-communications-professionnelles
https://www.life-global.org/fr/fr/course/124-leadership-efficace
https://www.life-global.org/fr/fr/course/337-%C3%A9conomie-circulaire
https://www.life-global.org/fr/fr/course/114-d%C3%A9marrer-une-petite-entreprise
https://www.life-global.org/fr/fr/course/323-l'entrep

In [97]:
df_courses

,Name,Duration,Pace,Language,Author,Code,Description,Category,Link
0,Des présentations efficaces,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR10,Comment puis-je créer un diaporama de présenta...,Communication,https://www.life-global.org/fr/fr/course/127-d...
1,Présenter des données,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR14,Quel est le moyen le plus efficace de présente...,Communication,https://www.life-global.org/fr/fr/course/125-p...
2,Les e-mails professionnels,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR11,Comment puis-je utiliser efficacement les e-ma...,Communication,https://www.life-global.org/fr/fr/course/126-l...
3,Communications professionnelles,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR26,Comment améliorer mes communications professio...,Communication,https://www.life-global.org/fr/fr/course/123-c...
4,Leadership efficace,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR23,Comment puis-je devenir un leader plus efficac...,Communication,https://www.life-global.org/fr/fr/course/124-l...
5,Économie circulaire,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR33,Comment puis-je concevoir mon entreprise afin ...,Créer / Innover,https://www.life-global.org/fr/fr/course/337-%...
6,Démarrer une petite entreprise,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR28,Comment puis-je créer une petite entreprise ?\...,Créer / Innover,https://www.life-global.org/fr/fr/course/114-d...
7,L'entrepreneuriat social,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR21,"Comment puis-je créer une entreprise durable, ...",Créer / Innover,https://www.life-global.org/fr/fr/course/323-l...
8,Moteurs de croissance,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR31,Qu’est-ce qu’un moteur de croissance et commen...,Créer / Innover,https://www.life-global.org/fr/fr/course/111-m...
9,Impression 3D,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE & Youth for Technology Foundation\r\n ...,HPL-FR29,Comment puis-je développer un site Internet po...,Créer / Innover,https://www.life-global.org/fr/fr/course/113-i...


In [98]:
language =  getAll(languageAll[0], languageBase[0])
df1 = df_categories
df2 = df_courses
df3 = df_lessons

new = pd.DataFrame(language[0], columns=['Name'])
df_categories = pd.concat([df_categories, new], ignore_index = True)

newi = pd.DataFrame(language[1])
df_courses = pd.concat([df_courses, newi], ignore_index = True)

newo = pd.DataFrame(language[2])
df_lessons = pd.concat([df_lessons, newo], ignore_index = True)

['Communication', 'Startup / Innovate', 'Finance', 'Marketing', 'Operations']
['https://www.life-global.org//categorylist/1-communication', 'https://www.life-global.org//categorylist/2-startup---innovate', 'https://www.life-global.org//categorylist/3-finance', 'https://www.life-global.org//categorylist/4-marketing', 'https://www.life-global.org//categorylist/5-operations']
https://www.life-global.org//course/12-effective-presentations
https://www.life-global.org//course/13-effective-leadership
https://www.life-global.org//course/10-business-email
https://www.life-global.org//course/9-business-communications-
https://www.life-global.org//course/11-presenting-data
https://www.life-global.org//course/333-circular-economy
https://www.life-global.org//course/17-starting-a-small-business
https://www.life-global.org//course/318-social-entrepreneurship
https://www.life-global.org//course/19-growth-engine-for-your-business
https://www.life-global.org//course/18-3d-printing
https://www.life-glob

In [99]:
language =  getAll(languageAll[2], languageBase[2])

df1 = df_categories
df2 = df_courses
df3 = df_lessons

new = pd.DataFrame(language[0], columns=['Name'])
df_categories = pd.concat([df_categories, new], ignore_index = True)

newi = pd.DataFrame(language[1])
df_courses = pd.concat([df_courses, newi], ignore_index = True)

newo = pd.DataFrame(language[2])
df_lessons = pd.concat([df_lessons, newo], ignore_index = True)

['الاتصالات', 'المشاريع الناشئة / الابتكار', 'المالية', 'تسويق', 'عمليات']
['https://www.life-global.org/ar/ar/categorylist/1-%D8%A7%D9%84%D8%A7%D8%AA%D8%B5%D8%A7%D9%84%D8%A7%D8%AA', 'https://www.life-global.org/ar/ar/categorylist/2-%D8%A7%D9%84%D9%85%D8%B4%D8%A7%D8%B1%D9%8A%D8%B9-%D8%A7%D9%84%D9%86%D8%A7%D8%B4%D8%A6%D8%A9---%D8%A7%D9%84%D8%A7%D8%A8%D8%AA%D9%83%D8%A7%D8%B1', 'https://www.life-global.org/ar/ar/categorylist/3-%D8%A7%D9%84%D9%85%D8%A7%D9%84%D9%8A%D8%A9', 'https://www.life-global.org/ar/ar/categorylist/4-%D8%AA%D8%B3%D9%88%D9%8A%D9%82', 'https://www.life-global.org/ar/ar/categorylist/5-%D8%B9%D9%85%D9%84%D9%8A%D8%A7%D8%AA']
https://www.life-global.org/ar/ar/course/213-%D8%A7%D9%84%D9%85%D8%AD%D8%A7%D8%B6%D8%B1%D8%A7%D8%AA-%D8%A7%D9%84%D9%81%D8%B9%D9%91%D8%A7%D9%84%D8%A9
https://www.life-global.org/ar/ar/course/212-%D8%A7%D9%84%D8%A8%D8%B1%D9%8A%D8%AF-%D8%A7%D9%84%D8%A5%D9%84%D9%83%D8%AA%D8%B1%D9%88%D9%86%D9%8A-%D9%84%D9%84%D8%B9%D9%85%D9%84
https://www.life-global.org/ar/a

In [101]:
language =  getAll(languageAll[3], languageBase[3])

df1 = df_categories
df2 = df_courses
df3 = df_lessons

new = pd.DataFrame(language[0], columns=['Name'])
df_categories = pd.concat([df_categories, new], ignore_index = True)

newi = pd.DataFrame(language[1])
df_courses = pd.concat([df_courses, newi], ignore_index = True)

newo = pd.DataFrame(language[2])
df_lessons = pd.concat([df_lessons, newo], ignore_index = True)

['संप्रेषण', 'स्टार्टअप / नवाचार', 'वित्त', 'विपणन', 'प्रचालन']
['https://www.life-global.org/hi/hi/categorylist/1-%E0%A4%B8%E0%A4%82%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A5%87%E0%A4%B7%E0%A4%A3', 'https://www.life-global.org/hi/hi/categorylist/2-%E0%A4%B8%E0%A5%8D%E0%A4%9F%E0%A4%BE%E0%A4%B0%E0%A5%8D%E0%A4%9F%E0%A4%85%E0%A4%AA---%E0%A4%A8%E0%A4%B5%E0%A4%BE%E0%A4%9A%E0%A4%BE%E0%A4%B0', 'https://www.life-global.org/hi/hi/categorylist/3-%E0%A4%B5%E0%A4%BF%E0%A4%A4%E0%A5%8D%E0%A4%A4', 'https://www.life-global.org/hi/hi/categorylist/4-%E0%A4%B5%E0%A4%BF%E0%A4%AA%E0%A4%A3%E0%A4%A8', 'https://www.life-global.org/hi/hi/categorylist/5-%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A4%9A%E0%A4%BE%E0%A4%B2%E0%A4%A8']
https://www.life-global.org/hi/hi/course/170-%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A4%AD%E0%A4%BE%E0%A4%B5%E0%A4%B6%E0%A4%BE%E0%A4%B2%E0%A5%80-%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A4%B8%E0%A5%8D%E0%A4%A4%E0%A5%81%E0%A4%A4%E0%A4%BF%E0%A4%AF%E0%A4%BE%E0%A4%81
https://www.life-global.org/hi/hi/course/169-%E0%A4%B5%E0%A5%8

In [102]:
language =  getAll(languageAll[4], languageBase[4])

df1 = df_categories
df2 = df_courses
df3 = df_lessons

new = pd.DataFrame(language[0], columns=['Name'])
df_categories = pd.concat([df_categories, new], ignore_index = True)

newi = pd.DataFrame(language[1])
df_courses = pd.concat([df_courses, newi], ignore_index = True)

newo = pd.DataFrame(language[2])
df_lessons = pd.concat([df_lessons, newo], ignore_index = True)

['Comunicação', 'Iniciar / Inovar', 'Finanças', 'Marketing', 'Operações']
['https://www.life-global.org/pt/pt/categorylist/1-comunica%C3%A7%C3%A3o', 'https://www.life-global.org/pt/pt/categorylist/2-iniciar---inovar', 'https://www.life-global.org/pt/pt/categorylist/3-finan%C3%A7as', 'https://www.life-global.org/pt/pt/categorylist/4-marketing', 'https://www.life-global.org/pt/pt/categorylist/5-opera%C3%A7%C3%B5es']
https://www.life-global.org/pt/pt/course/43-apresenta%C3%A7%C3%B5es-eficientes
https://www.life-global.org/pt/pt/course/44-lideran%C3%A7a-eficaz
https://www.life-global.org/pt/pt/course/45-apresenta%C3%A7%C3%A3o-de-dados
https://www.life-global.org/pt/pt/course/41-comunica%C3%A7%C3%A3o-de-neg%C3%B3cios
https://www.life-global.org/pt/pt/course/42-e-mail-comercial
https://www.life-global.org/pt/pt/course/338-economia-circular
https://www.life-global.org/pt/pt/course/48-como-come%C3%A7ar-uma-pequena-empresa
https://www.life-global.org/pt/pt/course/325-empreendedorismo-social
htt

In [104]:
language =  getAll(languageAll[5], languageBase[5])

df1 = df_categories
df2 = df_courses
df3 = df_lessons

new = pd.DataFrame(language[0], columns=['Name'])
df_categories = pd.concat([df_categories, new], ignore_index = True)

newi = pd.DataFrame(language[1])
df_courses = pd.concat([df_courses, newi], ignore_index = True)

newo = pd.DataFrame(language[2])
df_lessons = pd.concat([df_lessons, newo], ignore_index = True)

['Comunicación', 'Puesta en marcha / Innovar', 'Finanzas', 'Marketing', 'Operaciones']
['https://www.life-global.org/es/es/categorylist/1-comunicaci%C3%B3n', 'https://www.life-global.org/es/es/categorylist/2-puesta-en-marcha---innovar', 'https://www.life-global.org/es/es/categorylist/3-finanzas', 'https://www.life-global.org/es/es/categorylist/4-marketing', 'https://www.life-global.org/es/es/categorylist/5-operaciones']
https://www.life-global.org/es/es/course/139-presentaciones-efectivas
https://www.life-global.org/es/es/course/137-presentaci%C3%B3n-de-datos
https://www.life-global.org/es/es/course/138-correo-electr%C3%B3nico-comercial
https://www.life-global.org/es/es/course/136-liderazgo-eficaz
https://www.life-global.org/es/es/course/135-comunicaci%C3%B3n-empresarial
https://www.life-global.org/es/es/course/336-econom%C3%ADa-circular
https://www.life-global.org/es/es/course/143-empezar-un-peque%C3%B1o-negocio
https://www.life-global.org/es/es/course/324-emprendimiento-social
https:

In [113]:
language =  getAll(languageAll[6], languageBase[6])

df1 = df_categories
df2 = df_courses
df3 = df_lessons

new = pd.DataFrame(language[0], columns=['Name'])
df_categories = pd.concat([df_categories, new], ignore_index = True)

newi = pd.DataFrame(language[1])
df_courses = pd.concat([df_courses, newi], ignore_index = True)

newo = pd.DataFrame(language[2])
df_lessons = pd.concat([df_lessons, newo], ignore_index = True)

['沟通', '创业 / 创新', '财务', '营销', '运营']
['https://www.life-global.org/zh-Hans/zh-Hans/categorylist/1-%E6%B2%9F%E9%80%9A', 'https://www.life-global.org/zh-Hans/zh-Hans/categorylist/2-%E5%88%9B%E4%B8%9A---%E5%88%9B%E6%96%B0', 'https://www.life-global.org/zh-Hans/zh-Hans/categorylist/3-%E8%B4%A2%E5%8A%A1', 'https://www.life-global.org/zh-Hans/zh-Hans/categorylist/4-%E8%90%A5%E9%94%80', 'https://www.life-global.org/zh-Hans/zh-Hans/categorylist/5-%E8%BF%90%E8%90%A5']
https://www.life-global.org/zh-Hans/zh-Hans/course/74-%E7%BC%96%E5%88%B6%E4%BC%98%E7%A7%80%E7%9A%84%E6%BC%94%E7%A4%BA%E6%96%87%E7%A8%BF
https://www.life-global.org/zh-Hans/zh-Hans/course/76-%E5%B1%95%E7%A4%BA%E6%95%B0%E6%8D%AE
https://www.life-global.org/zh-Hans/zh-Hans/course/75-%E6%9C%89%E6%95%88%E9%A2%86%E5%AF%BC%E5%8A%9B
https://www.life-global.org/zh-Hans/zh-Hans/course/72-%E5%95%86%E5%8A%A1%E6%B2%9F%E9%80%9A
https://www.life-global.org/zh-Hans/zh-Hans/course/73-%E5%95%86%E5%8A%A1%E7%94%B5%E5%AD%90%E9%82%AE%E4%BB%B6
https://ww

In [114]:
language =  getAll(languageAll[7], languageBase[7])

df1 = df_categories
df2 = df_courses
df3 = df_lessons

new = pd.DataFrame(language[0], columns=['Name'])
df_categories = pd.concat([df_categories, new], ignore_index = True)

newi = pd.DataFrame(language[1])
df_courses = pd.concat([df_courses, newi], ignore_index = True)

newo = pd.DataFrame(language[2])
df_lessons = pd.concat([df_lessons, newo], ignore_index = True)

['Komunikasi', 'Memulai/Berinovasi', 'Keuangan', 'Pemasaran', 'Operasi']
['https://www.life-global.org/id/id/categorylist/1-komunikasi', 'https://www.life-global.org/id/id/categorylist/2-memulai-berinovasi', 'https://www.life-global.org/id/id/categorylist/3-keuangan', 'https://www.life-global.org/id/id/categorylist/4-pemasaran', 'https://www.life-global.org/id/id/categorylist/5-operasi']
https://www.life-global.org/id/id/course/291-presentasi-yang-efektif
https://www.life-global.org/id/id/course/289-email-bisnis
https://www.life-global.org/id/id/course/293-kepemimpinan-yang-efektif
https://www.life-global.org/id/id/course/288-komunikasi-bisnis
https://www.life-global.org/id/id/course/290-menyajikan-data
https://www.life-global.org/id/id/course/339-ekonomi-melingkar
https://www.life-global.org/id/id/course/296-memulai-usaha-kecil
https://www.life-global.org/id/id/course/319-kewirausahaan-sosial
https://www.life-global.org/id/id/course/298-mesin-pertumbuhan-untuk-bisnis-anda
https://www.

In [122]:
pd.set_option('display.max_rows', None)
df_categories
df_lessons
df_courses

,Name,Duration,Pace,Language,Author,Code,Description,Category,Link
0,Des présentations efficaces,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR10,Comment puis-je créer un diaporama de présenta...,Communication,https://www.life-global.org/fr/fr/course/127-d...
1,Présenter des données,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR14,Quel est le moyen le plus efficace de présente...,Communication,https://www.life-global.org/fr/fr/course/125-p...
2,Les e-mails professionnels,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR11,Comment puis-je utiliser efficacement les e-ma...,Communication,https://www.life-global.org/fr/fr/course/126-l...
3,Communications professionnelles,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR26,Comment améliorer mes communications professio...,Communication,https://www.life-global.org/fr/fr/course/123-c...
4,Leadership efficace,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR23,Comment puis-je devenir un leader plus efficac...,Communication,https://www.life-global.org/fr/fr/course/124-l...
5,Économie circulaire,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR33,Comment puis-je concevoir mon entreprise afin ...,Créer / Innover,https://www.life-global.org/fr/fr/course/337-%...
6,Démarrer une petite entreprise,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR28,Comment puis-je créer une petite entreprise ?\...,Créer / Innover,https://www.life-global.org/fr/fr/course/114-d...
7,L'entrepreneuriat social,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR21,"Comment puis-je créer une entreprise durable, ...",Créer / Innover,https://www.life-global.org/fr/fr/course/323-l...
8,Moteurs de croissance,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR31,Qu’est-ce qu’un moteur de croissance et commen...,Créer / Innover,https://www.life-global.org/fr/fr/course/111-m...
9,Impression 3D,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE & Youth for Technology Foundation\r\n ...,HPL-FR29,Comment puis-je développer un site Internet po...,Créer / Innover,https://www.life-global.org/fr/fr/course/113-i...


Limpieza de datos

In [118]:
df1 = df_categories
df2 = df_courses
df3 = df_lessons

In [132]:
df_courses["Language"] = df_courses["Language"].str.replace("\t", "")
df_courses.head(100)

,Name,Duration,Pace,Language,Author,Code,Description,Category,Link
0,Des présentations efficaces,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR10,Comment puis-je créer un diaporama de présenta...,Communication,https://www.life-global.org/fr/fr/course/127-d...
1,Présenter des données,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR14,Quel est le moyen le plus efficace de présente...,Communication,https://www.life-global.org/fr/fr/course/125-p...
2,Les e-mails professionnels,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR11,Comment puis-je utiliser efficacement les e-ma...,Communication,https://www.life-global.org/fr/fr/course/126-l...
3,Communications professionnelles,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR26,Comment améliorer mes communications professio...,Communication,https://www.life-global.org/fr/fr/course/123-c...
4,Leadership efficace,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR23,Comment puis-je devenir un leader plus efficac...,Communication,https://www.life-global.org/fr/fr/course/124-l...
5,Économie circulaire,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR33,Comment puis-je concevoir mon entreprise afin ...,Créer / Innover,https://www.life-global.org/fr/fr/course/337-%...
6,Démarrer une petite entreprise,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR28,Comment puis-je créer une petite entreprise ?\...,Créer / Innover,https://www.life-global.org/fr/fr/course/114-d...
7,L'entrepreneuriat social,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR21,"Comment puis-je créer une entreprise durable, ...",Créer / Innover,https://www.life-global.org/fr/fr/course/323-l...
8,Moteurs de croissance,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR31,Qu’est-ce qu’un moteur de croissance et commen...,Créer / Innover,https://www.life-global.org/fr/fr/course/111-m...
9,Impression 3D,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE & Youth for Technology Foundation\r\n ...,HPL-FR29,Comment puis-je développer un site Internet po...,Créer / Innover,https://www.life-global.org/fr/fr/course/113-i...


In [134]:
df_lessons["Type"] = df_lessons["Type"].str.replace("\r\n", "")
df_lessons.head(100)

,Type,Name,Course
0,Introduction,Histoire,Des présentations efficaces
1,Modules,Concept commercial,Des présentations efficaces
2,Modules,Compétence technologique,Des présentations efficaces
3,Modules,Sondage,Des présentations efficaces
4,Modules,Ressources,Des présentations efficaces
5,Introduction,Histoire,Présenter des données
6,Modules,Concept commercial,Présenter des données
7,Modules,Compétence technologique,Présenter des données
8,Modules,Sondage,Présenter des données
9,Modules,Ressources,Présenter des données


In [136]:
df_categories.to_csv("HPLife/CategoriesHP.csv",encoding='utf-8', index=False)
df_courses.to_csv("HPLife/CoursesHP.csv",encoding='utf-8', index=False)
df_lessons.to_csv("HPLife/LessonsHP.csv",encoding='utf-8', index=False)